<a href="https://colab.research.google.com/github/SeokcheonMoon/study_AIs/blob/main/docs/quests/MLs/SpineSurgeryList_FeatureEngine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc('font', family='NanumBarunGothic') # 혹은 다른 설치한 Nanum 폰트 사용
import scipy
import scipy.stats as stats

## 데이터
- 목표변수(재발여부)
- 설명변수(수치형 5개, 범주형 2개)

In [2]:
df_list = pd.read_csv("SpineSurgeryList.csv")
df_list

,Unnamed: 0,환자ID,Large Lymphocyte,Location of herniation,ODI,가족력,간질성폐질환,고혈압여부,과거수술횟수,당뇨여부,...,Modic change,PI,PT,Seg Angle(raw),Vaccum disc,골밀도,디스크단면적,디스크위치,척추이동척도,척추전방위증
0,0,1PT,22.8,3,51.0,0.0,0,0,0,0,...,3,51.6,36.6,14.4,0,-1.01,2048.50,4,Down,0
1,1,2PT,44.9,4,26.0,0.0,0,0,0,0,...,0,40.8,7.2,17.8,0,-1.14,1753.10,4,Up,0
2,2,3PT,53.0,3,NaN,0.0,0,0,1,0,...,0,67.5,27.3,10.2,0,-1.77,1749.00,4,Down,0
3,3,4PT,54.7,4,18.0,0.0,0,1,0,0,...,0,49.2,18.7,19.9,0,-1.58,1796.20,4,Down,0
4,4,5PT,53.8,4,NaN,0.0,0,1,0,0,...,0,58.8,14.7,5.2,0,-0.79,2021.48,4,Up,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1889,1889,1890PT,44.0,1,NaN,0.0,0,0,0,0,...,2,59.5,23.0,21.8,0,NaN,2652.20,5,Middle,0
1890,1890,1891PT,30.8,2,NaN,0.0,0,0,0,0,...,0,47.7,20.2,5.0,0,NaN,1795.80,4,Down,0
1891,1891,1892PT,36.7,3,NaN,0.0,0,0,0,0,...,0,44.6,15.0,17.4,1,NaN,2624.40,4,Down,0
1892,1892,1893PT,32.5,2,NaN,0.0,0,0,0,0,...,2,32.2,11.1,17.7,0,NaN,1972.50,5,Down,0


In [3]:
df_list.columns

Index(['Unnamed: 0', '환자ID', 'Large Lymphocyte', 'Location of herniation',
       'ODI', '가족력', '간질성폐질환', '고혈압여부', '과거수술횟수', '당뇨여부', '말초동맥질환여부', '빈혈여부',
       '성별', '스테로이드치료', '신부전여부', '신장', '심혈관질환', '암발병여부', '연령', '우울증여부', '입원기간',
       '입원일자', '종양진행여부', '직업', '체중', '퇴원일자', '헤모글로빈수치', '혈전합병증여부', '환자통증정도',
       '흡연여부', '통증기간(월)', '수술기법', '수술시간', '수술실패여부', '수술일자', '재발여부', '혈액형',
       '전방디스크높이(mm)', '후방디스크높이(mm)', '지방축적도', 'Instability', 'MF + ES',
       'Modic change', 'PI', 'PT', 'Seg Angle(raw)', 'Vaccum disc', '골밀도',
       '디스크단면적', '디스크위치', '척추이동척도', '척추전방위증'],
      dtype='object')

In [4]:
# 원하는 컬럼 빼내기
df_list.loc[:, ["재발여부","Large Lymphocyte","PI","PT","Seg Angle(raw)","디스크단면적","Location of herniation","디스크위치"]]           # target, feature(수치5, 범주2)

,재발여부,Large Lymphocyte,PI,PT,Seg Angle(raw),디스크단면적,Location of herniation,디스크위치
0,0,22.8,51.6,36.6,14.4,2048.50,3,4
1,0,44.9,40.8,7.2,17.8,1753.10,4,4
2,0,53.0,67.5,27.3,10.2,1749.00,3,4
3,0,54.7,49.2,18.7,19.9,1796.20,4,4
4,0,53.8,58.8,14.7,5.2,2021.48,4,4
...,...,...,...,...,...,...,...,...
1889,0,44.0,59.5,23.0,21.8,2652.20,1,5
1890,0,30.8,47.7,20.2,5.0,1795.80,2,4
1891,0,36.7,44.6,15.0,17.4,2624.40,3,4
1892,0,32.5,32.2,11.1,17.7,1972.50,2,5


In [5]:
df_list.loc[:, ["재발여부","Large Lymphocyte","PI","PT","Seg Angle(raw)","디스크단면적","Location of herniation","디스크위치"]].isnull().sum()  # 결측값 확인

재발여부                      0
Large Lymphocyte          0
PI                        4
PT                        4
Seg Angle(raw)            1
디스크단면적                    1
Location of herniation    0
디스크위치                     0
dtype: int64

In [6]:
df_list_extract = df_list.loc[:, ["재발여부","Large Lymphocyte","PI","PT","Seg Angle(raw)","디스크단면적","Location of herniation","디스크위치"]].dropna()  # 결측값 제거
df_list_extract # 새 변수에 담음

,재발여부,Large Lymphocyte,PI,PT,Seg Angle(raw),디스크단면적,Location of herniation,디스크위치
0,0,22.8,51.6,36.6,14.4,2048.50,3,4
1,0,44.9,40.8,7.2,17.8,1753.10,4,4
2,0,53.0,67.5,27.3,10.2,1749.00,3,4
3,0,54.7,49.2,18.7,19.9,1796.20,4,4
4,0,53.8,58.8,14.7,5.2,2021.48,4,4
...,...,...,...,...,...,...,...,...
1889,0,44.0,59.5,23.0,21.8,2652.20,1,5
1890,0,30.8,47.7,20.2,5.0,1795.80,2,4
1891,0,36.7,44.6,15.0,17.4,2624.40,3,4
1892,0,32.5,32.2,11.1,17.7,1972.50,2,5


In [7]:
df_list_extract.isnull().sum()  # 결측값 제거 확인

재발여부                      0
Large Lymphocyte          0
PI                        0
PT                        0
Seg Angle(raw)            0
디스크단면적                    0
Location of herniation    0
디스크위치                     0
dtype: int64

### 수치형 Scaling

In [8]:
from sklearn.preprocessing import MinMaxScaler
MMS = MinMaxScaler()
MMS.fit(df_list_extract[["Large Lymphocyte","PI","PT","Seg Angle(raw)","디스크단면적"]]) # 모델 학습

MinMaxScaler()

In [9]:
df_list_extract[["Large Lymphocyte_mms","PI_mms","PT_mms","Seg Angle(raw)_mms","디스크단면적_mms"]] = MMS.transform(df_list_extract[["Large Lymphocyte","PI","PT","Seg Angle(raw)","디스크단면적"]])
df_list_extract

,재발여부,Large Lymphocyte,PI,PT,Seg Angle(raw),디스크단면적,Location of herniation,디스크위치,Large Lymphocyte_mms,PI_mms,PT_mms,Seg Angle(raw)_mms,디스크단면적_mms
0,0,22.8,51.6,36.6,14.4,2048.50,3,4,0.246014,0.072564,0.795953,0.217256,0.436424
1,0,44.9,40.8,7.2,17.8,1753.10,4,4,0.497722,0.052824,0.300169,0.234927,0.293609
2,0,53.0,67.5,27.3,10.2,1749.00,3,4,0.589977,0.101627,0.639123,0.195426,0.291626
3,0,54.7,49.2,18.7,19.9,1796.20,4,4,0.609339,0.068178,0.494098,0.245842,0.314446
4,0,53.8,58.8,14.7,5.2,2021.48,4,4,0.599089,0.085725,0.426644,0.169439,0.423361
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1889,0,44.0,59.5,23.0,21.8,2652.20,1,5,0.487472,0.087004,0.566610,0.255717,0.728292
1890,0,30.8,47.7,20.2,5.0,1795.80,2,4,0.337130,0.065436,0.519393,0.168399,0.314253
1891,0,36.7,44.6,15.0,17.4,2624.40,3,4,0.404328,0.059770,0.431703,0.232848,0.714852
1892,0,32.5,32.2,11.1,17.7,1972.50,2,5,0.356492,0.037105,0.365936,0.234407,0.399681


### 범주형 Onehot Encoding

In [10]:
from sklearn.preprocessing import OneHotEncoder
OHE = OneHotEncoder()
OHE.fit(df_list_extract[["Location of herniation","디스크위치"]])   # 모델 학습

OneHotEncoder()

In [11]:
OHE.categories_               # Location of herniation 의 값들과 디스크위치 값 확인

[array([1, 2, 3, 4, 5]),
 array([ 1,  2,  3,  4,  5, 10, 11, 12, 23, 34, 35, 45])]

In [12]:
OHE.get_feature_names_out()   # 컬럼의 이름은 이렇게 될 것이다~

array(['Location of herniation_1', 'Location of herniation_2',
       'Location of herniation_3', 'Location of herniation_4',
       'Location of herniation_5', '디스크위치_1', '디스크위치_2', '디스크위치_3',
       '디스크위치_4', '디스크위치_5', '디스크위치_10', '디스크위치_11', '디스크위치_12',
       '디스크위치_23', '디스크위치_34', '디스크위치_35', '디스크위치_45'], dtype=object)

In [13]:
OHE.transform(df_list_extract[["Location of herniation","디스크위치"]])

<1888x17 sparse matrix of type '<class 'numpy.float64'>'
	with 3776 stored elements in Compressed Sparse Row format>

In [14]:
OHE_array = OHE.transform(df_list_extract[["Location of herniation","디스크위치"]]).toarray()          # 바로 위 내용을 numpy 방식으로 파일을 출력할 수 있음.          # but 카테고리가 붙어서 나오지 않음.
OHE_array

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [15]:
# dataframe 전환
df_OHE_array = pd.DataFrame(data=OHE_array, columns = OHE.get_feature_names_out())
df_OHE_array

,Location of herniation_1,Location of herniation_2,Location of herniation_3,Location of herniation_4,Location of herniation_5,디스크위치_1,디스크위치_2,디스크위치_3,디스크위치_4,디스크위치_5,디스크위치_10,디스크위치_11,디스크위치_12,디스크위치_23,디스크위치_34,디스크위치_35,디스크위치_45
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1883,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1884,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1885,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1886,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
# 행으로 concate
df_list_extract = pd.concat((df_list_extract.reset_index(drop=True),df_OHE_array.reset_index(drop=True)), axis = 1)          # reset_index(drop=True)
df_list_extract

,재발여부,Large Lymphocyte,PI,PT,Seg Angle(raw),디스크단면적,Location of herniation,디스크위치,Large Lymphocyte_mms,PI_mms,...,디스크위치_3,디스크위치_4,디스크위치_5,디스크위치_10,디스크위치_11,디스크위치_12,디스크위치_23,디스크위치_34,디스크위치_35,디스크위치_45
0,0,22.8,51.6,36.6,14.4,2048.50,3,4,0.246014,0.072564,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,44.9,40.8,7.2,17.8,1753.10,4,4,0.497722,0.052824,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,53.0,67.5,27.3,10.2,1749.00,3,4,0.589977,0.101627,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,54.7,49.2,18.7,19.9,1796.20,4,4,0.609339,0.068178,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,53.8,58.8,14.7,5.2,2021.48,4,4,0.599089,0.085725,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1883,0,44.0,59.5,23.0,21.8,2652.20,1,5,0.487472,0.087004,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1884,0,30.8,47.7,20.2,5.0,1795.80,2,4,0.337130,0.065436,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1885,0,36.7,44.6,15.0,17.4,2624.40,3,4,0.404328,0.059770,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1886,0,32.5,32.2,11.1,17.7,1972.50,2,5,0.356492,0.037105,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 정형화

In [17]:
# target 과 features 훈련컬럼 지정
target_train = df_list_extract["재발여부"]
features_train = df_list_extract.drop(columns = ["Large Lymphocyte","PI","PT","Seg Angle(raw)","디스크단면적","Location of herniation","디스크위치"])

In [18]:
target_train.shape, features_train.shape

((1888,), (1888, 23))

In [19]:
target_train

0       0
1       0
2       0
3       0
4       0
       ..
1883    0
1884    0
1885    0
1886    0
1887    0
Name: 재발여부, Length: 1888, dtype: int64

In [20]:
features_train          # 기존 컬럼이 사라진걸 확인할 수 있음.

,재발여부,Large Lymphocyte_mms,PI_mms,PT_mms,Seg Angle(raw)_mms,디스크단면적_mms,Location of herniation_1,Location of herniation_2,Location of herniation_3,Location of herniation_4,...,디스크위치_3,디스크위치_4,디스크위치_5,디스크위치_10,디스크위치_11,디스크위치_12,디스크위치_23,디스크위치_34,디스크위치_35,디스크위치_45
0,0,0.246014,0.072564,0.795953,0.217256,0.436424,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0.497722,0.052824,0.300169,0.234927,0.293609,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0.589977,0.101627,0.639123,0.195426,0.291626,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0.609339,0.068178,0.494098,0.245842,0.314446,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0.599089,0.085725,0.426644,0.169439,0.423361,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1883,0,0.487472,0.087004,0.566610,0.255717,0.728292,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1884,0,0.337130,0.065436,0.519393,0.168399,0.314253,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1885,0,0.404328,0.059770,0.431703,0.232848,0.714852,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1886,0,0.356492,0.037105,0.365936,0.234407,0.399681,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 모델

In [21]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model

LogisticRegression()

In [22]:
model.fit(features_train, target_train)

LogisticRegression()

In [29]:
model.predict(features_train[25:30])

array([0, 0, 0, 1, 0])

In [31]:
model.predict_proba(features_train[25:30])

array([[0.99455536, 0.00544464],
       [0.99518443, 0.00481557],
       [0.99439245, 0.00560755],
       [0.03463615, 0.96536385],
       [0.99461279, 0.00538721]])

## 평가

### 평가 수치

In [32]:
# 정확도
from sklearn.metrics import accuracy_score
target_train_predict = model.predict(features_train)
target_train_predict.shape

(1888,)

In [33]:
accuracy_score(target_train, target_train_predict)  # 1.0 이 나오면 모든 예측이 일치한다...?

1.0

### F1 score

In [34]:
from sklearn.metrics import classification_report
print(classification_report(target_train, target_train_predict))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1661
           1       1.00      1.00      1.00       227

    accuracy                           1.00      1888
   macro avg       1.00      1.00      1.00      1888
weighted avg       1.00      1.00      1.00      1888

